<a href="https://colab.research.google.com/github/hsollc/Life_at_Kangnam-basic_web_for_comments_and_chatting_etc-/blob/main/Copy_of_sbert%EC%9D%B4%EC%9A%A9%EC%9C%A0%EC%82%AC%ED%8C%90%EB%A1%80_%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])


In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

In [ ]:
!pwd

/content


In [ ]:
%cd "/content/drive/MyDrive/bert"

/content/drive/MyDrive/bert


In [ ]:
!pwd

/content/drive/MyDrive/bert


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.3 MB/s eta 0:00:00


In [ ]:
import transformers
transformers.__version__

'4.25.1'

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [ ]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

# train_data = pd.read_table('ratings_train.txt')
# test_data = pd.read_table('ratings_test.txt')

In [ ]:
# print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력

# print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력




In [ ]:
# train_data[:5] # 상위 5개 출력

# test_data[:5] # 상위 5개 출력



In [ ]:
data1 = pd.read_csv('dataset/부동산매매판례.csv')
data2 = pd.read_csv('dataset/약정금판례.csv')
data3 = pd.read_csv('dataset/저작권판례.csv')

In [ ]:
train_data=pd.concat([data1,data2,data3])[['판결요지']]

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data.reset_index(drop=True)
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

# test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
# test_data = test_data.reset_index(drop=True)
# print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인

print(len(train_data))

# print(len(test_data))

False
2084


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))


[101, 9356, 11018, 31605, 31605, 110589, 71568, 118913, 11018, 9576, 119281, 9786, 79940, 23811, 40364, 9520, 23160, 102]
['보', '##는', '##내', '##내', '그대로', '들어', '##맞', '##는', '예', '##측', '카', '##리스', '##마', '없는', '악', '##역']


'[CLS] 보는내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

In [ ]:
for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
  print(tokenizer.decode(elem))

[ C L S ]
보
# # 는
# # 내
# # 내
그 대 로
들 어
# # 맞
# # 는
예
# # 측
카
# # 리 스
# # 마
없 는
악
# # 역
[ S E P ]


In [ ]:
print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

['전', '##율', '##을', '일', '##으', '##키는', '영화', '.', '다시', '보고', '##싶', '##은', '영화']
[101, 9665, 119183, 10622, 9641, 119185, 66815, 42428, 119, 25805, 98199, 119088, 10892, 42428, 102]


In [ ]:
txt = '타인의 소송에서 사실을 증언하는 증인이 그 증언을 조건으로 그 소송의 일방 당사자 등으로부터 통상적으로 용인될 수 있는 수준예컨대 증인에게 일당 및 여비가 지급되기는 하지만 증인이 증언을 위하여 법원에 출석함으로써 입게 되는 손해에는 미치지 못하는 경우 그러한 손해를 전보하여 주는 정도을 넘어서는 대가를 제공받기로 하는 약정은 국민의 사법참여행위가 대가와 결부됨으로써 사법작용의 불가매수성 내지 대가무관성이 본질적으로 침해되는 경우로서 반사회적 법률행위에 해당하여 무효라고 할 것이다 이는 증언거부권이 있는 증인이 그 증언거부권을 포기하고 증언을 하는 경우라고 하여 달리 볼 것이 아니다'

In [ ]:
print(tokenizer.tokenize(txt))

print(tokenizer.encode(txt))

['타', '##인의', '소', '##송', '##에서', '사', '##실을', '증', '##언', '##하는', '증', '##인이', '그', '증', '##언', '##을', '조', '##건', '##으로', '그', '소', '##송', '##의', '일', '##방', '당', '##사', '##자', '등으로', '##부터', '통', '##상', '##적으로', '용', '##인', '##될', '수', '있는', '수', '##준', '##예', '##컨', '##대', '증', '##인', '##에게', '일', '##당', '및', '여', '##비', '##가', '지', '##급', '##되', '##기는', '하지만', '증', '##인이', '증', '##언', '##을', '위하여', '법', '##원에', '출', '##석', '##함', '##으로', '##써', '입', '##게', '되는', '손', '##해', '##에는', '미', '##치', '##지', '못', '##하는', '경우', '그', '##러', '##한', '손', '##해를', '전', '##보', '##하여', '주', '##는', '정도', '##을', '넘', '##어', '##서는', '대', '##가를', '제', '##공', '##받', '##기로', '하는', '약', '##정', '##은', '국', '##민', '##의', '사', '##법', '##참', '##여', '##행', '##위', '##가', '대', '##가와', '결', '##부', '##됨', '##으로', '##써', '사', '##법', '##작', '##용', '##의', '불', '##가', '##매', '##수', '##성', '내', '##지', '대', '##가', '##무', '##관', '##성이', '본', '##질', '##적으로', '침', '##해', '##되는', '경우', '##로서', '반', '##사', '##회', '##적', '법

In [ ]:
temp = tokenizer.encode(txt)

In [ ]:
set(temp)

{101,
 102,
 8881,
 8909,
 8924,
 8996,
 9008,
 9067,
 9069,
 9290,
 9294,
 9309,
 9316,
 9321,
 9341,
 9358,
 9359,
 9368,
 9405,
 9448,
 9450,
 9460,
 9519,
 9539,
 9565,
 9603,
 9641,
 9645,
 9665,
 9672,
 9678,
 9689,
 9705,
 9706,
 9768,
 9783,
 9845,
 9879,
 9928,
 9955,
 10459,
 10622,
 10739,
 10892,
 11018,
 11102,
 11287,
 11467,
 11489,
 12030,
 12178,
 12310,
 12453,
 12508,
 12945,
 12965,
 13374,
 13764,
 13767,
 14153,
 14423,
 14523,
 14646,
 14801,
 14863,
 14871,
 15303,
 15891,
 16605,
 17022,
 17138,
 17655,
 18622,
 19855,
 20595,
 21928,
 23969,
 24190,
 24683,
 24974,
 25347,
 25549,
 26212,
 27487,
 28000,
 28467,
 29455,
 29935,
 30005,
 30873,
 32537,
 32775,
 33768,
 33904,
 36553,
 37321,
 37568,
 38709,
 40958,
 41521,
 42337,
 46216,
 48036,
 48345,
 48418,
 48533,
 48599,
 51076,
 53371,
 54780,
 54867,
 56789,
 57952,
 59330,
 59894,
 62675,
 68010,
 71439,
 73131,
 73450,
 78139,
 79611,
 83380,
 86475,
 88350,
 91621,
 92278,
 92318,
 92413,
 96279,
 1

In [ ]:
for elem in tokenizer.encode(txt):
  print(tokenizer.decode(elem))

[ C L S ]
타
# # 인 의
소
# # 송
# # 에 서
사
# # 실 을
증
# # 언
# # 하 는
증
# # 인 이
그
증
# # 언
# # 을
조
# # 건
# # 으 로
그
소
# # 송
# # 의
일
# # 방
당
# # 사
# # 자
등 으 로
# # 부 터
통
# # 상
# # 적 으 로
용
# # 인
# # 될
수
있 는
수
# # 준
# # 예
# # 컨
# # 대
증
# # 인
# # 에 게
일
# # 당
및
여
# # 비
# # 가
지
# # 급
# # 되
# # 기 는
하 지 만
증
# # 인 이
증
# # 언
# # 을
위 하 여
법
# # 원 에
출
# # 석
# # 함
# # 으 로
# # 써
입
# # 게
되 는
손
# # 해
# # 에 는
미
# # 치
# # 지
못
# # 하 는
경 우
그
# # 러
# # 한
손
# # 해 를
전
# # 보
# # 하 여
주
# # 는
정 도
# # 을
넘
# # 어
# # 서 는
대
# # 가 를
제
# # 공
# # 받
# # 기 로
하 는
약
# # 정
# # 은
국
# # 민
# # 의
사
# # 법
# # 참
# # 여
# # 행
# # 위
# # 가
대
# # 가 와
결
# # 부
# # 됨
# # 으 로
# # 써
사
# # 법
# # 작
# # 용
# # 의
불
# # 가
# # 매
# # 수
# # 성
내
# # 지
대
# # 가
# # 무
# # 관
# # 성 이
본
# # 질
# # 적 으 로
침
# # 해
# # 되 는
경 우
# # 로 서
반
# # 사
# # 회
# # 적
법
# # 률
# # 행
# # 위 에
해 당
# # 하 여
무
# # 효
# # 라 고
할
것 이 다
이 는
증
# # 언
# # 거
# # 부
# # 권
# # 이
있 는
증
# # 인 이
그
증
# # 언
# # 거
# # 부
# # 권 을
포
# # 기
# # 하 고
증
# # 언
# # 을
하 는
경 우
# # 라 고
하 여
달 리
볼
것 이
아
# # 니 다
[ S E P ]


In [ ]:
print(tokenizer.decode(101))

print(tokenizer.decode(102))

[ C L S ]
[ S E P ]


In [ ]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)


[CLS] : 101
[SEP] : 102


In [ ]:
print(tokenizer.pad_token, ':', tokenizer.pad_token_id)

[PAD] : 0


In [ ]:
max_seq_len = 300
encoded_result = tokenizer.encode(txt, max_length=max_seq_len, pad_to_max_length=True,truncation=True)
print(encoded_result)
print('길이 :', len(encoded_result))

[101, 9845, 48418, 9448, 119057, 11489, 9405, 83380, 9705, 48036, 12178, 9705, 56789, 8924, 9705, 48036, 10622, 9678, 71439, 11467, 8924, 9448, 119057, 10459, 9641, 42337, 9067, 12945, 13764, 86475, 17655, 9879, 14871, 17022, 9603, 12030, 59330, 9460, 13767, 9460, 54867, 96279, 119297, 14423, 9705, 12030, 26212, 9641, 21928, 9316, 9565, 29455, 11287, 9706, 37568, 118800, 46216, 32775, 9705, 56789, 9705, 48036, 10622, 68010, 9341, 108280, 9768, 40958, 48533, 11467, 73131, 9645, 14153, 54780, 9450, 14523, 15303, 9309, 18622, 12508, 9290, 12178, 28467, 8924, 30873, 11102, 9450, 92278, 9665, 30005, 13374, 9689, 11018, 107657, 10622, 9008, 12965, 37321, 9069, 92318, 9672, 28000, 118965, 62675, 23969, 9539, 16605, 10892, 8909, 36553, 10459, 9405, 33768, 119251, 29935, 25549, 19855, 11287, 9069, 79611, 8881, 14646, 118801, 11467, 73131, 9405, 33768, 38709, 24974, 10459, 9368, 11287, 100372, 15891, 17138, 8996, 12508, 9069, 11287, 32537, 20595, 53371, 9358, 48599, 17022, 9783, 14523, 24683, 28

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# 세그멘트 인풋
print([0]*max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# 마스크 인풋
valid_num = len(tokenizer.encode(txt))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
def convert_examples_to_features(examples, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
       
        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
        # token_type_id는 세그먼트 임베딩을 위한 것으로 이번 예제는 문장이 1개이므로 전부 0으로 통일.
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    
    return (input_ids, attention_masks, token_type_ids)

In [ ]:
### 문장 임베딩 sentence_transformers

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=280ec653c6b182294b1b2f42a163e2799e2091b63f2b3943a26fb196af2ba3d6
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
import urllib.request
import pandas as pd

from sentence_transformers import SentenceTransformer

In [ ]:
train_data=train_data[train_data['판결요지']!=' '].reset_index(drop=True)
train_data.head()

,판결요지
0,甲이 하남시장에게 용도를 ‘농산물 직거래용 가설점포’로 하고 존치기간을 3년 이...
1,[1] 구 도시 및 주거환경정비법(2013. 12. 24. 법률 제12116호로 개...
2,[1] 당사자는 소장에 기재된 표시 및 청구의 내용과 원인사실을 합리적으로 해석하여...
3,[1] 계약의 해석은 계약 내용을 명확하게 확정하는 것으로서 당사자 사이에 계약의 ...
4,"[1] 국세징수법상 체납처분 유예와 압류해제, 납세담보에 관한 규정의 체계와 문..."


In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')


Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
train_data.apply(lambda row: print(row))

0         甲이 하남시장에게 용도를 ‘농산물 직거래용 가설점포’로 하고 존치기간을 3년 이...
1       [1] 구 도시 및 주거환경정비법(2013. 12. 24. 법률 제12116호로 개...
2       [1] 당사자는 소장에 기재된 표시 및 청구의 내용과 원인사실을 합리적으로 해석하여...
3       [1] 계약의 해석은 계약 내용을 명확하게 확정하는 것으로서 당사자 사이에 계약의 ...
4         [1] 국세징수법상 체납처분 유예와 압류해제, 납세담보에 관한 규정의 체계와 문...
                              ...                        
1301    제작신고된 영화의 상영을 위한 검열신청에 대하여 행정청은 검열합격 여부를 결정할 수...
1302    구 저작권법(57.1.28. 법률 제432호) 제16조의 이른바 저작자의 명예와 성...
1303    (1) 문교부장관의 「검인정」은 검인정의 공고가 된 도서가 각급 학교의 교과용 도서...
1304     본조의 문서는 본법 제316조 각호에 의하여 문서제출의무 있는 것이라야 하며 이와...
1305    원고 간행의 위 옥편에는 저작권이 인정되지만, 국내에서 간행된 모든 옥편의 모태가 ...
Name: 판결요지, Length: 1306, dtype: object


판결요지    None
dtype: object

In [ ]:
train_data['embedding'] = train_data['판결요지'].apply(lambda row: model.encode(row))

train_data.head()

,판결요지,embedding
0,甲이 하남시장에게 용도를 ‘농산물 직거래용 가설점포’로 하고 존치기간을 3년 이...,"[-0.31147346, -0.073408544, 0.85874754, -0.598..."
1,[1] 구 도시 및 주거환경정비법(2013. 12. 24. 법률 제12116호로 개...,"[0.01767302, 0.41717035, 0.25802636, -0.034593..."
2,[1] 당사자는 소장에 기재된 표시 및 청구의 내용과 원인사실을 합리적으로 해석하여...,"[-0.68212134, 0.19737451, 0.6434947, 0.2059368..."
3,[1] 계약의 해석은 계약 내용을 명확하게 확정하는 것으로서 당사자 사이에 계약의 ...,"[-0.29900867, 0.39915514, 0.91172725, 0.172642..."
4,"[1] 국세징수법상 체납처분 유예와 압류해제, 납세담보에 관한 규정의 체계와 문...","[-0.25778466, 0.7339301, -0.28581306, 0.075974..."


In [ ]:
emb = np.array(train_data['embedding'])

In [ ]:
np.save('embed.npy',emb)

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_similar_answer(input):
    embedding = model.encode(input)
    train_data['score'] = train_data['embedding'].apply(lambda x: cos_sim(x, embedding))
    return train_data.loc[train_data['score'].argsort()[::-1]].head()


In [ ]:
txt = '타인의 소송에서 사실을 증언하는 증인이 그 증언을 조건으로 그 소송의 일방 당사자 등으로부터 통상적으로 용인될 수 있는 수준예컨대 증인에게 일당 및 여비가 지급되기는 하지만 증인이 증언을 위하여 법원에 출석함으로써 입게 되는 손해에는 미치지 못하는 경우 그러한 손해를 전보하여 주는 정도을 넘어서는 대가를 제공받기로 하는 약정은 국민의 사법참여행위가 대가와 결부됨으로써 사법작용의 불가매수성 내지 대가무관성이 본질적으로 침해되는 경우로서 반사회적 법률행위에 해당하여 무효라고 할 것이다 이는 증언거부권이 있는 증인이 그 증언거부권을 포기하고 증언을 하는 경우라고 하여 달리 볼 것이 아니다'

In [ ]:
txt = "[1] 구 개인정보 보호법(2020. 2. 4. 법률 제16930호로 일부 개정되기 전의 것, 이하 같다) 제18조 제2항 제7호는 개인정보처리자가 ‘범죄의 수사와 공소의 제기 및 유지를 위하여 필요한 경우’에는 정보주체 또는 제3자의 이익을 부당하게 침해할 우려가 있는 때를 제외하고는 개인정보를 목적 외의 용도로 이용하거나 이를 제3자에 제공할 수 있음을 규정하였으나, 이는 ‘개인정보처리자’가 ‘공공기관’인 경우에 한정될 뿐 법인·단체·개인 등의 경우에는 적용되지 아니한다(구 개인정보 보호법 제18조 제2항 단서, 제2조 제5호 및 제6호). 또한 구 개인정보 보호법 제18조 제2항 제2호에서 정한 ‘다른 법률에 특별한 규정이 있는 경우’란 그 문언 그대로 개별 법률에서 개인정보의 제공이 허용됨을 구체적으로 명시한 경우로 한정하여 해석하여야 하므로, 형사소송법 제199조 제2항과 같이 수사기관이 공무소 기타 공사단체에 조회하여 필요한 사항의 보고를 요구할 수 있는 포괄적인 규정은 이에 해당하지 아니한다. 만일 형사소송법 제199조 제2항이 구 개인정보 보호법 제18조 제2항 제2호에서 정한 ‘다른 법률에 특별한 규정이 있는 경우’에 포함된다면, 구 개인정보 보호법 제18조 제2항 제7호에서 수사기관으로 하여금 공공기관에 한정하여 일정한 제한 아래 개인정보를 제공받을 수 있도록 한 입법 취지·목적을 몰각시킬 뿐만 아니라 헌법상 영장주의 및 적법절차의 원칙을 잠탈할 가능성이 크기 때문이다. [2] 개인정보 보호법 제59조 제2호의 의무주체인 ‘개인정보를 처리하거나 처리하였던 자’는 제2조 제5호의 ‘개인정보처리자’에 한정되지 않고, 업무상 알게 된 제2조 제1호의 ‘개인정보’를 제2조 제2호의 방법으로 ‘처리’하거나 ‘처리’하였던 자를 포함한다.[3] 형사소송법 제312조 제5항은 피고인 또는 피고인이 아닌 자가 수사과정에서 작성한 진술서의 증거능력에 관하여 형사소송법 제312조 제1항부터 제4항까지 준용하도록 규정하고 있으므로, 검사 또는 사법경찰관이 피고인이 아닌 자의 진술을 기재한 조서의 증거능력이 인정되려면 ‘적법한 절차와 방식에 따라 작성된 것’이어야 한다는 법리가 피고인이 아닌 자가 수사과정에서 작성한 진술서의 증거능력에 관하여도 적용된다. 한편 검사 또는 사법경찰관이 피의자가 아닌 자의 출석을 요구하여 조사하는 경우에는 피의자를 조사하는 경우와 마찬가지로 조사장소에 도착한 시각, 조사를 시작하고 마친 시각, 그 밖에 조사과정의 진행경과를 확인하기 위하여 필요한 사항을 조서에 기록하거나 별도의 서면에 기록한 후 수사기록에 편철하도록 하는 등 조사과정을 기록하게 한 형사소송법 제221조 제1항, 제244조의4 제1항, 제3항의 취지는 수사기관이 조사과정에서 피조사자로부터 진술증거를 취득하는 과정을 투명하게 함으로써 그 과정에서의 절차적 적법성을 제도적으로 보장하려는 것이다. 따라서 수사기관이 수사에 필요하여 피의자가 아닌 자로부터 진술서를 작성·제출받는 경우에도 그 절차는 준수되어야 하므로, 피고인이 아닌 자가 수사과정에서 진술서를 작성하였지만 수사기관이 조사과정의 진행경과를 확인하기 위하여 필요한 사항을 그 진술서에 기록하거나 별도의 서면에 기록한 후 수사기록에 편철하는 등 적절한 조치를 취하지 아니하여 형사소송법 제244조의4 제1항, 제3항에서 정한 절차를 위반한 경우에는, 그 진술증거 취득과정의 절차적 적법성의 제도적 보장이 침해되지 않았다고 볼 만한 특별한 사정이 없는 한 ‘적법한 절차와 방식’에 따라 수사과정에서 진술서가 작성되었다고 할 수 없어 증거능력을 인정할 수 없다.이러한 형사소송법 규정 및 문언과 그 입법 목적 등에 비추어 보면, 형사소송법 제312조 제5항의 적용대상인 ‘수사과정에서 작성한 진술서’란 수사가 시작된 이후에 수사기관의 관여 아래 작성된 것이거나, 개시된 수사와 관련하여 수사과정에 제출할 목적으로 작성한 것으로, 작성 시기와 경위 등 여러 사정에 비추어 그 실질이 이에 해당하는 이상 명칭이나 작성된 장소 여부를 불문한다."

In [ ]:
return_similar_answer(txt)

,판결요지,embedding,score
1088,"[1] 헌법 제10조, 제17조의 규정은 개인의 사생활이 타인에 의해 침해되거나 함...","[-0.1943559, 0.8320719, -0.1303346, 0.14887638...",0.728082
924,[1] 구 정보통신망 이용촉진 및 정보보호 등에 관한 법률(2018. 12. 2...,"[0.5263359, 0.03007739, 0.28703815, -0.1061015...",0.714298
951,"[1] 공소사실의 기재는 범죄의 일시, 장소와 방법을 명시하여 사실을 특정할 수...","[-0.21458238, 0.25058204, 0.344163, 0.41663337...",0.690015
1119,"[1] ""나홀로""로 구성된 등록서비스표가 변호사업 등 그 지정서비스업과 관련하여 전...","[-0.108177684, 0.61882776, 0.5958349, -0.02585...",0.683682
972,"저작권법 제104조 제1항, 저작권법 시행령 제46조 제1항의 규정 취지는 저작물 ...","[-0.11676365, 0.6199733, 0.2754198, 0.25344798...",0.679678
